In [ ]:
import pandas as pd
import seaborn as sns
import autokeras as ak
from sklearn.model_selection import train_test_split

# --- 1. Tải và chuẩn bị dữ liệu ---
# AutoKeras CÓ THỂ xử lý dữ liệu thô (NaNs và Categorical)

# Tải bộ dữ liệu Titanic
data = sns.load_dataset('titanic')

# Chỉ lấy các cột cơ bản
data = data[['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']]

# Tách X (features) và y (target)
# Lưu ý: X là pandas DataFrame, y là pandas Series. AutoKeras thích điều này.
X = data.drop('survived', axis=1)
y = data['survived']

# Chia dữ liệu: 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    stratify=y,
                                                    random_state=42)

print(f"Kích thước tập Train: {X_train.shape}")
print(f"Kích thước tập Test: {X_test.shape}")


# --- 2. Thiết lập và Huấn luyện (Train) AutoKeras ---

print("\nBắt đầu khởi tạo AutoKeras...")
# 1. Khởi tạo
# 'StructuredDataClassifier' được thiết kế riêng cho dữ liệu bảng
# max_trials: Số lượng kiến trúc mạng nơ-ron khác nhau sẽ được thử.
#             Đây là thông số quan trọng nhất để cân bằng thời gian/chất lượng.
#             10 trials là đủ nhanh cho demo.
clf = ak.StructuredDataClassifier(
    max_trials=10, 
    project_name='titanic_autokeras',
    objective='val_accuracy', # Tối ưu theo accuracy trên tập validation
    metrics=['accuracy', 'roc_auc'], # Báo cáo cả 2 chỉ số này
    overwrite=True
)

# 2. Huấn luyện (Train)
# AutoKeras sẽ tự động trích 1 phần của X_train làm tập validation.
# 'epochs': Số epochs để huấn luyện cho MỖI trial. 
#          Để 'None' (mặc định) AutoKeras sẽ tự tìm, hoặc đặt 20-50 cho demo.
print("Bắt đầu huấn luyện (Tìm kiếm kiến trúc)...")
clf.fit(X_train, y_train, epochs=20)


# --- 3. Đánh giá (Evaluate) trên tập Test ---
# Sử dụng phương thức .evaluate() của Keras

print("\nĐánh giá mô hình tốt nhất trên tập Test...")
# Hàm evaluate trả về [loss, accuracy, roc_auc] (theo thứ tự của metrics)
results = clf.evaluate(X_test, y_test)
loss = results[0]
acc = results[1]
auc = results[2] # Chỉ số 'roc_auc'

print("--- KẾT QUẢ ĐÁNH GIÁ (AutoKeras) ---")
print(f"Loss trên tập Test: {loss:.4f}")
print(f"Accuracy trên tập Test: {acc:.4f}")
print(f"ROC-AUC trên tập Test: {auc:.4f}")


# --- 4. (Quan trọng) Xuất ra mô hình Keras tốt nhất ---
# Đây là tính năng rất mạnh: bạn có thể lấy ra mô hình Keras/TensorFlow
# đã được huấn luyện đầy đủ.

print("\nĐang xuất mô hình Keras tốt nhất...")
best_model = clf.export_model()

# In cấu trúc của mạng nơ-ron mà nó đã tìm thấy
best_model.summary()

# Bạn có thể lưu lại mô hình này để sử dụng sau
# best_model.save('autokeras_titanic_model.tf', save_format="tf")